In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from ultralytics import YOLO
import os
import werkzeug
import threading
app = Flask(__name__)
CORS(app)  # CORS 적용

app.config['UPLOAD_FOLDER'] = './uploads'

# 모델 불러오기
best_model = YOLO('./data/runs/detect/train2/weights/best.pt')

@app.route('/', methods=['POST'])
def predict():
    print("predict 함수 실행")
    imagefile = request.files['image']
    filename = werkzeug.utils.secure_filename(imagefile.filename)
    filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    imagefile.save(filepath)

    # 예측시키기
    results = best_model(filepath, save=True, conf=0.3)

    # 예측 결과 저장을 위한 리스트
    predictions = []

    for result in results:
        for box in result.boxes:
            class_name = result.names[int(box.cls[0])]
            predictions.append({
                'class': class_name,
                'message': '50개 입니다.' if class_name == '50_towel_counts' else '50개가 아닙니다.'
            })

    # 예측 결과가 없을 경우 오류 메시지 반환
    if not predictions:
        return jsonify({
            'error': 'No valid class found.'
        }), 400

    # 예측 결과 반환
    return jsonify(predictions), 200

 #별도의 스레드에서 Flask 앱 실행
def run_app(app):
    app.run(use_reloader=False)  # reloader를 비활성화

threading.Thread(target=run_app, args=(app,)).start()